# AOI 資料準備

1. [引言](#引言)
2. [下載並載入資料](#下載並載入資料)
3. [資料清理](#資料清理)
4. [分割訓練與驗證資料](#分割訓練與驗證資料)
5. [產生lst資料](#產生lst資料)
6. [產生RecordIO資料](#產生RecordIO資料)
7. [上傳資料至S3](#上傳資料至S3)

## 引言

訓練模型前，必須先準備好資料。我們將透過此筆記本將資料準備好。主要步驟包含：
1. 下載已標記好的瑕疵影像資料
2. 將資料中的錯誤修正
3. 將整份資料分成兩部份用於訓練與驗證
4. 將資料轉換成 lst 與 RecordIO 格式
5. 上傳 RecordIO 資料至 S3

如想準備自己的瑕疵影像資料，一般來說需要蒐集圖片與標記圖片。常見蒐集圖片方式為上網搜尋或自己拍照，標記圖片則可使用常見的工具如 [LabelImg](https://github.com/tzutalin/labelImg)。

雖然 Amazon SageMaker 除了 RecordIO 也可使用圖片和 JSON 格式訓練模型，依 Amazon 官方效率上的建議，在此使用 RecordIO 格式。

## 下載並載入資料

In [ ]:
# download compressed file
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UZkJgdF2MRzF3NY0XDrvI0HaaXky3iiz' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UZkJgdF2MRzF3NY0XDrvI0HaaXky3iiz" -O NEU-DET.zip && rm -rf /tmp/cookies.txt

In [ ]:
# decompress
!unzip -oqq NEU-DET.zip

In [ ]:
# get paths of annotations

import os

img_dir = 'NEU-DET/IMAGES'
xml_dir = 'NEU-DET/ANNOTATIONS'
xmls = []

# Exclude certain type of defect to simplify the problem
exclude_class = 'crazing'

for root, dirs, files in os.walk(xml_dir):
    for file in files:
        if file.endswith('.xml') and exclude_class not in file:
            xmls.append(os.path.join(root, file))

## 資料清理

In [ ]:
# fix errors in the annotations

with open(os.path.join(xml_dir, 'pitted_surface_107.xml'), 'r+') as f:
    contents = f.readlines()
    contents.insert(5, '  </source>\n')
    f.truncate(0)
    f.seek(0)
    f.writelines(contents)

with open(os.path.join(xml_dir, 'rolled-in_scale_145.xml'), 'r+') as f:
    contents = f.readlines()
    contents[36] = '</annotation>'
    f.truncate(0)
    f.seek(0)
    f.writelines(contents)

with open(os.path.join(xml_dir, 'scratches_228.xml'), 'r+') as f:
    contents = f.readlines()
    contents[37] = '		<name>patches</name>\n'
    f.truncate(0)
    f.seek(0)
    f.writelines(contents)

## 分割訓練與驗證資料

In [ ]:
# for each defect class we use 250 images for training and 50 for validation

num_classes = 5
num_train_per_class = 250
num_train = num_classes * num_train_per_class

xmls.sort()

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))

xmls_by_classes = list(split(xmls, num_classes))
train_xmls = []
test_xmls = []
for xml_class in xmls_by_classes:
    train_xmls.extend(xml_class[:num_train_per_class])
    test_xmls.extend(xml_class[num_train_per_class:])

## 產生lst資料

有關 lst 檔案格式的更多資訊，可參考 [AWS RecordIO Notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/object_detection_pascalvoc_coco/object_detection_recordio_format.ipynb)。

In [ ]:
import xml.etree.ElementTree as ET
import json

img_idx = -1
class_map = {}
not_exist_paths = []
trainlines = []
testlines = []
lstlines = []

for xml in train_xmls+test_xmls:
    outputs = []
    img_idx += 1
    #print(img_idx, xml)
    #if img_idx > 10:
    #    break
    tree = ET.parse(xml)
    root = tree.getroot()
    filename = root.find('filename').text
    width = float(root.find('size').find('width').text)
    height = float(root.find('size').find('height').text)
    img_path = os.path.join(img_dir, filename)
    outputs.extend([img_idx, 2, 5])

    if not os.path.exists(img_path):
        if os.path.exists(img_path + '.jpg'):
            img_path = img_path + '.jpg'
            #print('Found match: ' + img_path)
        else:
            not_exist_paths.append(img_path)
            #print('Error: ' + img_path + ' does not exist and has no exact match')

    for o in root.findall('object'):
        classname = o.find('name').text
        
        if classname=='paches':
            print(xml)

        if not classname in class_map:
            class_map[classname] = len(class_map.keys())

        bndbox = o.find('bndbox')
        xmin = float(bndbox.find('xmin').text) / width
        ymin = float(bndbox.find('ymin').text) / height
        xmax = float(bndbox.find('xmax').text) / width
        ymax = float(bndbox.find('ymax').text) / height
        outputs.extend([class_map[classname], xmin, ymin, xmax, ymax])
    outputs.append(img_path)
    lstlines.append('\t'.join(map(str, outputs))+'\n')

with open('train.lst', 'w') as f:
    f.writelines(lstlines[:num_train])
with open('val.lst', 'w') as f:
    f.writelines(lstlines[num_train:])
with open('class_index.txt', 'w') as f:
    f.write(json.dumps(class_map, indent=4, separators=(',', ': ')))
print('Finished processing {0} images'.format(len(xmls)))

## 產生RecordIO資料

有關 RecordIO 檔案格式的更多資訊，可參考 [AWS RecordIO Notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/object_detection_pascalvoc_coco/object_detection_recordio_format.ipynb)。

In [ ]:
# download conversion tool
!wget https://raw.githubusercontent.com/apache/incubator-mxnet/master/tools/im2rec.py

In [ ]:
# do conversion
!python im2rec.py --pack-label --num-thread 4 train.lst .
!python im2rec.py --pack-label --num-thread 4 val.lst .

## 上傳資料至S3

In [ ]:
bucket = 'sagemaker-aoi-xxx'
prefix = 'DEMO-AOI'

import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()

train_channel = prefix + '/train'
validation_channel = prefix + '/validation'

s3_train_data = 's3://{}/{}'.format(bucket, train_channel)
s3_validation_data = 's3://{}/{}'.format(bucket, validation_channel)

# upload RecordIO files
sess.upload_data(path='train.rec', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='val.rec', bucket=bucket, key_prefix=validation_channel)

In [ ]:
# upload lst files and images

sess.upload_data(path='train.lst', bucket=bucket, key_prefix=train_channel)
sess.upload_data(path='val.lst', bucket=bucket, key_prefix=validation_channel)

# for l in lstlines[:num_train]:
#     items = l.split('\t')
#     img_path = items[-1].strip()
#     sess.upload_data(path=img_path, bucket=bucket, key_prefix=train_channel)

for l in lstlines[num_train:]:
    items = l.split('\t')
    img_path = items[-1].strip()
    sess.upload_data(path=img_path, bucket=bucket, key_prefix=validation_channel)

print('Finished uploading files')